In [209]:
from pymongo import MongoClient
from sklearn.cluster import KMeans
import pandas as pd
from bson.objectid import ObjectId
import IPython
from ipywidgets import interact, interactive, fixed, interact_manual

In [69]:
pd.set_option('display.max_rows', None)

In [6]:
# connect to db
client = MongoClient('mongodb://mongo:27017/')
db = client.music_analysis
songs = db.songs


In [215]:
print(songs.count())

TypeError: count() takes exactly one argument (0 given)

In [98]:
df = pd.DataFrame(list(songs.find()))

In [157]:
df.sample()

,_id,filename,highleveldanceabilityalldanceable,highlevelgenre_electronicallambient,highlevelgenre_electronicalldnb,highlevelgenre_electronicallhouse,highlevelgenre_electronicalltechno,highlevelgenre_electronicalltrance,highlevelgenre_rosamericaallcla,highlevelgenre_rosamericaalldan,...,highlevelmood_partyvalue,highlevelmood_relaxedvalue,highlevelmood_sadvalue,highlevelmoods_mirexvalue,highleveltimbrevalue,highlevelvoice_instrumentalvalue,tonalchords_key,tonalchords_scale,tonalkey_krumhanslkey,tonalkey_krumhanslscale
5363,5e4efba47ac2d042041f6001,./../../music/DJ Shadow/In-Fluxuations/10 The ...,0.994655,0.05939,0.057232,0.573034,0.178376,0.131967,0.000648,0.0045,...,party,not_relaxed,not_sad,Cluster5,dark,instrumental,C#,major,C,major


In [136]:
# drop invalid data
# TODO map strings to numbers
dfCleaned = df.drop([
    # string values
    '_id',
    'filename',
    'highlevelmood_partyvalue',
    'highlevelmood_relaxedvalue',
    'highlevelmood_sadvalue',
    'highlevelmoods_mirexvalue',
    'highleveltimbrevalue',
    'highlevelgenre_rosamericavalue',
    'highlevelgenre_electronicvalue',
    'highlevelmood_acousticvalue',
    'highlevelmood_aggressivevalue',
    'highleveldanceabilityvalue',
    'highlevelmood_electronicvalue',
    'highlevelmood_happyvalue',
    'highlevelvoice_instrumentalvalue',
    'tonalchords_key',
    'tonalchords_scale',
    'tonalkey_krumhanslkey',
    'tonalkey_krumhanslscale',
], axis=1)

# normalize all data to average 100
for (columnName, columnData) in dfCleaned.iteritems():
    #print(columnName)
    normalizeMultiplier = 100 / columnData.mean()
    #print(normalizeMultiplier)
    dfCleaned[columnName] *= normalizeMultiplier

dfCleaned.sample()

,highleveldanceabilityalldanceable,highlevelgenre_electronicallambient,highlevelgenre_electronicalldnb,highlevelgenre_electronicallhouse,highlevelgenre_electronicalltechno,highlevelgenre_electronicalltrance,highlevelgenre_rosamericaallcla,highlevelgenre_rosamericaalldan,highlevelgenre_rosamericaallhip,highlevelgenre_rosamericaalljaz,...,lowleveldynamic_complexity,rhythmbeats_count,rhythmbeats_loudnessmedian,rhythmbpm,rhythmdanceability,rhythmonset_rate,tonalchords_changes_rate,tonalchords_number_rate,tonalchords_strengthmedian,tonaltuning_frequency
7657,143.118437,7.380668,27.142156,246.735733,7.578603,9.306606,3.368943,54.020961,263.688329,4.054547,...,117.141733,58.457155,139.91251,86.425813,92.305304,119.798375,131.318154,121.770211,102.152915,99.193076


In [137]:
# cluster data
# TODO how to determine # of clusters?
kmeans = KMeans(n_clusters=250, random_state=0).fit(dfCleaned)
labels = kmeans.predict(dfCleaned)

In [138]:
# add labels to results
dfCleaned.insert(0, 'group', labels)
dfCleaned.insert(0, 'song', df['filename'])
dfCleaned.insert(0, 'id', df['_id'])

In [218]:
# show a cluster
cluster = dfCleaned.query('group == 1')
cluster

,id,song,group,highleveldanceabilityalldanceable,highlevelgenre_electronicallambient,highlevelgenre_electronicalldnb,highlevelgenre_electronicallhouse,highlevelgenre_electronicalltechno,highlevelgenre_electronicalltrance,highlevelgenre_rosamericaallcla,...,lowleveldynamic_complexity,rhythmbeats_count,rhythmbeats_loudnessmedian,rhythmbpm,rhythmdanceability,rhythmonset_rate,tonalchords_changes_rate,tonalchords_number_rate,tonalchords_strengthmedian,tonaltuning_frequency
137,5e4cceae4814f92fad32a1c6,./../../music/MonkeyRobot/Earshot Ep/04 Shango...,1,141.625645,38.733593,45.380905,96.020537,512.215622,181.437052,17.482328,...,94.182748,170.564134,146.300265,105.121144,113.185852,145.163747,87.748458,36.012571,102.672166,99.193076
169,5e4cd10a4814f92fad32a1e6,./../../music/Groove Armada/Goodbye Country (H...,1,131.654141,79.205400,99.130488,127.940883,268.113817,30.938616,17.414468,...,93.986736,103.069195,200.686292,58.945180,98.107316,80.817333,70.821831,25.753916,107.560248,99.193076
541,5e4ce94a4814f92fad32a35a,./../../music/The Cinematic Orchestra/Man With...,1,113.103720,67.694299,17.333218,135.853005,433.668111,19.638465,6.822219,...,113.851796,144.604542,124.541985,131.543264,99.734717,84.277055,103.429597,58.720035,99.098082,99.193076
813,5e4cfc574814f92fad32a46a,./../../music/DJ Krush/Strictly Turntablized/0...,1,106.330194,23.634732,20.979711,185.506674,438.880671,4.912361,5.788752,...,115.435456,75.571257,118.145608,73.754541,135.717376,97.515318,186.764352,75.578685,84.687071,99.537450
815,5e4cfc674814f92fad32a46c,./../../music/DJ Krush/Strictly Turntablized/0...,1,79.617313,44.112625,17.342807,152.369316,560.869023,2.782335,79.674202,...,135.695079,33.651323,135.715293,78.814625,137.169034,85.461868,34.834967,27.233682,81.835035,100.287699
2216,5e4d68d64814f92fad32a9e5,./../../music/Kruder & Dorfmeister/DJ-Kicks_ K...,1,141.599380,109.601065,48.447582,98.781409,258.360716,45.158048,1.054843,...,68.949123,119.029537,327.768967,87.338955,103.356921,137.926934,93.786865,59.977775,94.556587,102.394957
2259,5e4d6c114814f92fad32aa10,./../../music/The Herbaliser/Very Mercenary/15...,1,110.392352,110.528830,53.436348,82.483301,344.442867,60.272041,12.398265,...,73.017917,159.411124,101.857159,126.347533,103.813932,122.920986,129.583831,69.176409,94.483677,98.621767
2266,5e4d6c884814f92fad32aa17,./../../music/The Herbaliser/Remedies/13 Wrong...,1,98.073987,116.580851,68.067724,55.171780,654.295329,20.113015,2.516559,...,104.293990,93.839118,110.686110,76.643072,100.790912,124.752732,192.191527,71.530763,82.375160,98.507895
2394,5e4d77ff4814f92fad32aa97,./../../music/DJ Krush & 近藤等則/Ki-Oku/05 Sun Is...,1,116.889512,71.830666,36.981180,135.606707,272.050346,50.806614,30.284157,...,79.909835,187.293649,205.791770,120.559187,125.805076,102.835936,93.456522,41.336945,102.852792,103.584732
3749,5e4ddb3d4814f92fad32afe2,./../../music/Massive Attack/Singles 90_98/19 ...,1,72.312591,115.675077,108.862805,65.317219,480.907759,33.071372,19.414638,...,111.673052,76.340430,75.235982,80.040128,108.892492,83.428545,101.127492,87.232249,113.733293,101.922885


In [219]:
# collect songs
playlist = []
for index, song in cluster.iterrows():
    playlist.append(song['song'])
    #IPython.display.display(IPython.display.Audio(song['song'], rate=44100))
playlist

['./../../music/MonkeyRobot/Earshot Ep/04 Shango.mp3',
 './../../music/Groove Armada/Goodbye Country (Hello Nightclub)/10 Edge Hill.mp3',
 './../../music/The Cinematic Orchestra/Man With a Movie Camera/15 Drunken Tune.mp3',
 './../../music/DJ Krush/Strictly Turntablized/03 Fucked-Up Pendulum.mp3',
 './../../music/DJ Krush/Strictly Turntablized/05 The Loop.mp3',
 './../../music/Kruder & Dorfmeister/DJ-Kicks_ Kruder & Dorfmeister/11 High Noon.mp3',
 './../../music/The Herbaliser/Very Mercenary/15 The Missing Suitcase.mp3',
 './../../music/The Herbaliser/Remedies/13 Wrong Place.mp3',
 './../../music/DJ Krush & 近藤等則/Ki-Oku/05 Sun Is Shining.mp3',
 './../../music/Massive Attack/Singles 90_98/19 Home of the Whale.mp3',
 './../../music/Rob Dougan/Furious Angels/22 Nothing at All (instrumental).mp3',
 './../../music/Boards of Canada/Twoism/04 Basefree.mp3',
 "./../../music/Atjazz/Get Physical Music Presents_ Body Language Volume 14 By Andhim/12 It's Complete (Chateau Flight Darkside Mix).mp3",

In [220]:
# show player
def play(song):
    IPython.display.display(IPython.display.Audio(song, rate=44100))
    
interact(play, song=playlist);

interactive(children=(Dropdown(description='song', options=('./../../music/MonkeyRobot/Earshot Ep/04 Shango.mp…